In [1]:
import torch 

_=torch.tensor([0.2126,0.7152,0.0722],names=['c'])

C:\Users\DC\AppData\Local\Temp\ipykernel_14992\2053401103.py:3: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ..\c10/core/TensorImpl.h:1903.)
  _=torch.tensor([0.2126,0.7152,0.0722],names=['c'])


In [2]:
img_t=torch.randn(3,5,5)
weights=torch.tensor([0.2126,0.7152,0.0722])

In [3]:
batch_t=torch.randn(2,3,5,5)


In [4]:
img_gray_naive=img_t.mean(-3)
batch_gray_naive=batch_t.mean(-3)
img_gray_naive.shape ,batch_gray_naive.shape 

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [7]:
img_t.shape 

torch.Size([3, 5, 5])

In [6]:

batch_t.shape

torch.Size([2, 3, 5, 5])

In [8]:
weights.shape

torch.Size([3])

In [5]:
#torch.unsqueeze(input, dim) → Tensor
#Returns a new tensor with a dimension of size one inserted at the specified position.
#The returned tensor shares the same underlying data with this tensor.

unsqueezed_weights=weights.unsqueeze(-1).unsqueeze_(-1)
img_weights=(img_t *unsqueezed_weights)
batch_weights=(batch_t *unsqueezed_weights)

img_gray_weighted=img_weights.sum(-3)
batch_gray_weighted=batch_weights.sum(-3)

batch_weights.shape ,batch_t.shape ,unsqueezed_weights.shape 







(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [9]:
unsqueezed_weights.shape

torch.Size([3, 1, 1])

In [10]:
#torch.einsum(equation, *operands) → Tensor[SOURCE]
#Sums the product of the elements of the input operands along dimensions specified using a notation
# based on the Einstein summation convention.

img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [11]:
weighted_named=torch.tensor([0.2126,0.7152,0.0722],names=['channels'])
weighted_named


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [12]:
#efine_names(*names)[SOURCE]
#Refines the dimension names of self according to names.

img_named=img_t.refine_names(...,'channels','rows','columns')
batch_named=batch_t.refine_names(...,'channels','rows','columns')

print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)


img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [13]:
#align_as(other) → Tensor
#Permutes the dimensions of the self tensor to match the dimension order in the other tensor, adding size-one dims for any new names.

weights_aligned=weighted_named.align_as(img_named)
weights_aligned.shape ,weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [15]:
gray_named=(img_named* weights_aligned).sum('channels')

gray_named.shape ,gray_named.names


(torch.Size([5, 5]), ('rows', 'columns'))

In [16]:
try:
    gray_named=(img_named[...,:3]*weighted_named).sum('channels')
except Exception as e:
    print(e)


Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [17]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))